In [1]:
import re
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [2]:
agua = pd.read_csv('../data/consumo-agua.csv')

In [3]:
agua.head()

,Geo Point,Geo Shape,consumo_total_mixto,anio,nomgeo,consumo_prom_dom,consumo_total_dom,alcaldia,colonia,consumo_prom_mixto,consumo_total,consumo_prom,consumo_prom_no_dom,bimestre,consumo_total_no_dom,gid,indice_des
0,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",159.72,2019,Gustavo A. Madero,42.566364,468.23,GUSTAVO A. MADERO,7 DE NOVIEMBRE,53.24,631.00,42.066667,3.05,3,3.05,57250,ALTO
1,"19.4552601937,-99.1126617526","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,35.936667,107.81,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,115.13,28.782500,7.32,3,7.32,57253,MEDIO
2,"19.4557195871,-99.1135822797","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,24.586000,122.93,GUSTAVO A. MADERO,7 DE NOVIEMBRE,0.00,197.96,32.993333,75.03,3,75.03,57255,POPULAR
3,"19.4596467168,-99.1044693641","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",0.00,2019,Gustavo A. Madero,0.000000,0.00,GUSTAVO A. MADERO,NUEVA TENOCHTITLAN,0.00,253.53,84.510000,84.51,3,253.53,57267,BAJO
4,"19.4741606185,-99.1467497317","{""type"": ""MultiPolygon"", ""coordinates"": [[[[-9...",56.72,2019,Azcapotzalco,67.436250,539.49,AZCAPOTZALCO,PROHOGAR,56.72,839.35,76.304545,121.57,3,243.14,57330,BAJO


In [8]:
agua = agua.astype({"bimestre":'category', "indice_des":'category'}) 

In [10]:
# Dividimos variables por tipo de datos
vars_type = agua.dtypes
vars_type = pd.DataFrame(vars_type, columns = ['tipo'])
vars_type['variable']=vars_type.index

# variables numericas
variables_int = vars_type.loc[vars_type["tipo"] == "int64"]
variables_float = vars_type.loc[vars_type["tipo"] == "float64"]
variables_numericas = variables_int.append(variables_float, ignore_index=True)
lista_numericas = list(variables_numericas['variable'])

# variables fecha
variables_date = vars_type.loc[vars_type["tipo"] == "Date"]
lista_date = list(variables_date['variable'])

# variables categoricas
variables_category = vars_type.loc[vars_type["tipo"] == "category"]
lista_category = list(variables_category['variable'])

# variables texto
variables_texto = vars_type.loc[vars_type["tipo"] == "object"]
lista_texto = list(variables_texto['variable'])

In [23]:
def genera_profiling_de_categorias(df):
    # Obtenemos los estadísticos de la columna si es catagorica
    lista_perfilamiento_categorico = ['tipo','numero de categorias', 'numero de observaciones',
                                      'observaciones nulas','% observaciones nulas', 'valores unicos',
                                      'moda1/veces/porcentaje','moda2/veces/porcentaje','moda3/veces/porcentaje']
    datos_dataframe_profiling_categoricos = {'metrica':lista_perfilamiento_categorico}
    dataframe_profiling_categoricas = pd.DataFrame(data=datos_dataframe_profiling_categoricos)
    for col in lista_category:
        # tipo de dato
        vars_type_cat = pd.DataFrame(vars_type)
        #vars_type_cat
        df_tipo = pd.DataFrame(data=vars_type_cat.loc[vars_type_cat["variable"] == col])
        tipo_dato=df_tipo['tipo'][0]        

        # Obtenemos las métricas relevantes
        descr_col = df[col]
        descr_col = pd.DataFrame(descr_col)
        descr_col['metrica']=descr_col.index
        descr_col.columns=['valor','metrica']
        
        #Numero de categorias
        
        num_categorias=descr_col.nunique()["valor"]
        
        #Numero de observaciones
        
        num_observaciones=len(descr_col)
        
        #Valores nulos
        
        num_obs_nulas=agua[col].isna().sum()
        
        #%Valores nulos
        
        por_obs_nulas=num_obs_nulas/num_observaciones
        
        # valor de las categorias
        valores_unicos = list(df[col].unique())
        
        # generamos tabla para las modas
        tabla_importantes = CreaTablaConteoPorcentaje(df,str(col),True)
        tabla_importantes.columns = ['conteo','porcentaje']
        
        moda1 = tabla_importantes.index[0]
        veces1 = tabla_importantes['conteo'][0]
        porcentaje1 = tabla_importantes['porcentaje'][0]
        datos_moda1 = [moda1,veces1,porcentaje1]
        
        moda2 = tabla_importantes.index[1]
        veces2 = tabla_importantes['conteo'][1]
        porcentaje2 = tabla_importantes['porcentaje'][1]
        datos_moda2 = [moda2,veces2,porcentaje2]
        
        moda3 = tabla_importantes.index[2]
        veces3 = tabla_importantes['conteo'][2]
        porcentaje3 = tabla_importantes['porcentaje'][2]
        datos_moda3 = [moda3,veces3,porcentaje3]

        datos_variable = [tipo_dato,num_categorias,num_observaciones,num_obs_nulas,por_obs_nulas,
                          valores_unicos,datos_moda1,datos_moda2,datos_moda3]
        dataframe_profiling_categoricas[col]=datos_variable
    return dataframe_profiling_categoricas

In [24]:
genera_profiling_de_categorias(agua)

,metrica,bimestre,indice_des
0,tipo,category,category
1,numero de categorias,3,4
2,numero de observaciones,71102,71102
3,observaciones nulas,0,0
4,% observaciones nulas,0,0
5,valores unicos,"[3, 1, 2]","[ALTO, MEDIO, POPULAR, BAJO]"
6,moda1/veces/porcentaje,"[2, 23942, 33.67%]","[BAJO, 29248, 41.14%]"
7,moda2/veces/porcentaje,"[3, 23822, 33.5%]","[POPULAR, 16539, 23.26%]"
8,moda3/veces/porcentaje,"[1, 23338, 32.82%]","[ALTO, 15516, 21.82%]"


In [13]:
def CreaTablaConteoPorcentaje(df, nomColumna, booleanNA):
    """
    Esta función crea la tabla con información sobre los conteos y el porcentaje al que corresponden del total de los datos.
    
    ==========
    * Args:
      - df: el data frame que contiene los parquets importados.
      - nomColumna: El nombre de la columna sobre la que se quiere realizar la tabla.
      - booleanNA: Indicador booleano que indica si se requiere que se muestren los NA's en la tabla.
    * Return:
      - Data Frame: entrega el data frame con los la categoría de la columna RESPUESTA modificada.
    ==========
    Ejemplo:
      >>df = CreaTablaConteoPorcentaje(df, 'RESPUESTA', True)
    
    """
    
    df_resultado = df[nomColumna].value_counts(dropna=booleanNA)
    df_resultado = pd.DataFrame(data=df_resultado)
    
    #obteniendo los porcentajes
    df_resultado['porcentaje'] = df[nomColumna].value_counts(dropna=booleanNA, normalize=True).mul(100).round(2).astype(str)+'%'
    
    return df_resultado

In [14]:
tabla = CreaTablaConteoPorcentaje(agua,'bimestre',True)
tabla

,bimestre,porcentaje
2,23942,33.67%
3,23822,33.5%
1,23338,32.82%


In [15]:
tabla.columns = ['conteo','porcentaje']
tabla

,conteo,porcentaje
2,23942,33.67%
3,23822,33.5%
1,23338,32.82%


In [16]:
tabla['conteo'][0]

23942

In [20]:
tabla.index[0]

2